In [1]:
import time
import torch
import timeit
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import namedtuple
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

%matplotlib inline
np.random.seed(2018)

Mounted at /content/drive


In [2]:
train = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntrain.npy',allow_pickle=True)
train_labels = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntrain_labels.npy',allow_pickle=True)
val = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/nval.npy',allow_pickle=True)
val_labels = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/nval_labels.npy',allow_pickle=True)
ntest = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntest.npy',allow_pickle=True)
phones = np.loadtxt("C:/Users/malji/Google Drive/Colab Notebooks/Hw3/phones.txt", dtype=str)


In [3]:
class PhonesDataset(data.Dataset):
    def __init__(self, x,y,k):       
        self.x = x
        self.y = y
        self.k = k
        self.inputs = []
        self.lab = np.array([])
        self.idx = []
        self._init_dataset()
        
    def __len__(self):
        return len(self.lab)

    def __getitem__(self,index):
          i = self.idx[index]
          return torch.from_numpy(np.concatenate(self.inputs[i-self.k:i+self.k+1],axis=0)).float(),torch.tensor(self.lab[index]).long()
    
    def _init_dataset(self):
        idx=0
        for i in range(len(self.x)):
            start_time = time.time()#######
            s = len(self.x[i])
            x=np.pad(self.x[i],((self.k, self.k), (0, 0)), 'constant', constant_values=0)

            idx += 2*self.k
            for j in range(s):           
              self.idx+=[idx-self.k]
              idx += 1

            self.inputs+=list(x)  
            self.lab = np.append(self.lab,self.y[i])
            end_time = time.time()#######  
        return np.array(self.inputs), self.lab

In [ ]:
# # Dataset
# class PhonesDataset(data.Dataset):
#   def __init__(self, X,Y,k): 
#     self.X = X
#     self.Y = Y
#     self.k = k
#     self.samples = []
#     self.labels = []
#     self.length = []
#     self._init_dataset()
#     self.ind = np.arange(self.length[-1])
#     km = [self.k*(2*i+1) for i in range(len(self.length))]
        
#     b = 0
#     for i in range(self.length[-1]):
#         if i == self.length[b]:
#             b = b+1
#             self.ind[i] = self.ind[i] + km[b]
#         else:
#             self.ind[i] = self.ind[i] + km[b]

#   def __len__(self):
#     print(len(self.samples),len(self.labels))
#     return len(self.labels)

#   def __getitem__(self,index):
#     X = np.concatenate((self.samples[self.ind[index]-self.k:self.ind[index]+ self.k+1]),axis=0)
#     labels = self.labels[index]
#     return torch.from_numpy(X).float(),torch.tensor(labels).long()
    
#   def _init_dataset(self):
#     s = 0
#     for i in range(len(self.X)):
#         p = np.pad(self.X[i], ((self.k, self.k), (0, 0)), 'constant', constant_values=0)
#         s = s + len(self.X[i])
#         self.length.append(s)
#         self.samples = self.samples + list(p)
#         self.labels = self.labels + list(self.Y[i]) 

#     return np.array(self.samples), np.array(self.labels)

In [5]:
class TestDataset(data.Dataset):
    def __init__(self, x,k):       
        self.x = x
        self.k = k
        self.inputs = []
        self.idx = []
        self._init_dataset()
        
    def __len__(self):
        return len(self.idx)

    def __getitem__(self,index):
          idx = self.idx[index]
          return torch.from_numpy(np.concatenate(self.inputs[idx-self.k:idx+self.k+1],axis=0)).float()
    
    def _init_dataset(self):
        idx=0
        for i in range(len(self.x)):
            start_time = time.time()#######
            s = len(self.x[i])
            x=np.pad(self.x[i],((self.k, self.k), (0, 0)), 'constant', constant_values=0)

            idx += 2*self.k
            for j in range(s):           
              self.idx+=[idx-self.k]
              idx += 1

            self.inputs+=list(x)  
            end_time = time.time()#######
            if i==1000:##########
              print('Time: ',end_time - start_time, 's') #######    
        return np.array(self.inputs)

In [ ]:
# class TestDataset(data.Dataset):
#     def __init__(self, X,k): 
#       self.X = X
#       self.k = k
#       self.samples = []
#       self.length = []
#       self._init_dataset()
#       self.ind = np.arange(self.length[-1])
#       km = [self.k*(2*i+1) for i in range(len(self.length))]
      
#       b = 0
#       for i in range(self.length[-1]):
#           if i == self.length[b]:
#               b = b+1
#               self.ind[i] = self.ind[i] + km[b]
#           else:
#               self.ind[i] = self.ind[i] + km[b]
        

#     def __len__(self):
#       print(len(self.samples),self.length[-1])
#       return self.length[-1]

#     def __getitem__(self,index):
#       X = np.concatenate((self.samples[self.ind[index]-self.k:self.ind[index]+ self.k+1]),axis=0)
#       return torch.from_numpy(X).float()
    
#     def _init_dataset(self):
#       s = 0
#       for i in range(len(self.X)):
#           p = np.pad(self.X[i], ((self.k, self.k), (0, 0)), 'constant', constant_values=0)
#           s = s + len(self.X[i])
#           self.length.append(s)
#           self.samples = self.samples + list(p)
         
#       return np.array(self.samples)

In [6]:
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0 
    
# Training
start_time = time.time()
train_dataset = PhonesDataset(train, train_labels,13)
train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True)
train_loader = data.DataLoader(train_dataset, **train_loader_args)
print("taken time: %s seconds ---" % (time.time() - start_time))

taken time: 815.9661796092987 seconds ---


In [7]:
# Validation
start_time = time.time()
num_workers = 8 
val_dataset = PhonesDataset(val, val_labels,13)
val_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True)
val_loader = data.DataLoader(val_dataset, **val_loader_args)
print("taken time: %s seconds ---" % (time.time() - start_time))

taken time: 7.250003814697266 seconds ---


In [9]:
# Testing
start_time = time.time()
test_dataset = TestDataset(ntest,13)
test_loader_args = dict(shuffle=False, batch_size=1, num_workers=num_workers, pin_memory=True)
test_loader = data.DataLoader(test_dataset, **test_loader_args)
print("taken time: %s seconds ---" % (time.time() - start_time))

Time:  0.0 s
taken time: 2.0421011447906494 seconds ---


In [ ]:
# def save_data(loader):
#     print('saving data...')
    
#     for i, (x, y) in enumerate(loader):
        
#         x=x.view(-1, 13).numpy()
#         y = y.numpy()
#         if i==0:
#             data = np.array(x)
#             label = np.array(y)
#         else:     
#             data = np.concatenate((data, x))
#             label = np.concatenate((label, y))
#     return data, label

In [ ]:
# data, labels = save_data(train_loader)
# np.save('/content/drive/My Drive/Colab Notebooks/train_data.npy', data)
# np.save('/content/drive/My Drive/Colab Notebooks/train_labels.npy', labels)

# data, labels = save_data(val_loader)
# np.save('/content/drive/My Drive/Colab Notebooks/valid_data.npy', data)
# np.save('/content/drive/My Drive/Colab Notebooks/valid_labels.npy', labels)

# data, labels = save_data(test_loader)
# np.save('/content/drive/My Drive/Colab Notebooks/test_data.npy', data)
# np.save('/content/drive/My Drive/Colab Notebooks/test_labels.npy', labels)


In [12]:
# # loading data
# train = np.load('/content/drive/My Drive/Colab Notebooks/Hw3/ntrain.npy',allow_pickle=True)
# train_labels = np.load('/content/drive/My Drive/Colab Notebooks/Hw3/ntrain_labels.npy',allow_pickle=True)
# val = np.load('/content/drive/My Drive/Colab Notebooks/Hw3/nval.npy',allow_pickle=True)
# val_labels = np.load('/content/drive/My Drive/Colab Notebooks/Hw3/nval_labels.npy',allow_pickle=True)
# test = np.load('/content/drive/My Drive/Colab Notebooks/Hw3/ntest.npy',allow_pickle=True)
# phones = np.loadtxt("/content/drive/My Drive/Colab Notebooks/Hw3/phones.txt", dtype=str)
# train_size = train.shape[0]*1721



val_size = val.shape[0]

test_size = test.shape[0]

batch_size = 32

# train_data = PhonesDataset(np.load('/content/drive/My Drive/Colab Notebooks/Hw3/ntrain.npy',allow_pickle=True),
#                            np.load('/content/drive/My Drive/Colab Notebooks/Hw3/ntrain_labels.npy',allow_pickle=True))
# train_loader = torch.utils.data.DataLoader(train_data, 
#                                            batch_size=batch_size,
#                                            shuffle=True
#                                            )
# val_data = PhonesDataset(np.load('/content/drive/My Drive/Colab Notebooks/Hw3/nval.npy',allow_pickle=True),
#                          np.load('/content/drive/My Drive/Colab Notebooks/Hw3/nval_labels.npy',allow_pickle=True))
# val_loader = torch.utils.data.DataLoader(val_data, 
#                                          batch_size=batch_size,
#                                          shuffle=False
#                                            )
# test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

# def save_data(loader):
#     print('saving data...')
    
#     for i, (x, y) in enumerate(loader):
        
#         x=x.view(-1, 13).numpy()
#         y = y.numpy()
#         if i==0:
#             data = np.array(x)
#             label = np.array(y)
#         else:     
#             data = np.concatenate((data, x))
#             label = np.concatenate((label, y))
#     return data, label

# # I didn't add the code of the saving 

# #data, labels = save_data(train_loader)
# #np.save('./data/train_data.npy', data)
# #np.save('./data/train_labels.npy', labels)

# #data, labels = save_data(val_loader)
# #np.save('./data/valid_data.npy', data)
# #np.save('./data/valid_labels.npy', labels)

# #data, labels = save_data(test_loader)
# #np.save('./data/test_data.npy', data)
# #np.save('./data/test_labels.npy', labels)






Metric = namedtuple('Metric', ['loss', 'train_error', 'val_error'])

In [13]:
class PhonesModel(nn.Module):
    # try changing 32 to 128
    def __init__(self):
        super(PhonesModel, self).__init__()
        self.fc1 = nn.Linear(351, 512 )
        self.bnorm1 = nn.BatchNorm1d(512 )
        self.dp1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(512 , 256 )
        self.bnorm2 = nn.BatchNorm1d(256 )
        self.dp2 = nn.Dropout(p=0.1)
        self.fc3 = nn.Linear(256, 256 )
        self.bnorm3 = nn.BatchNorm1d(256 )
        self.dp3 = nn.Dropout(p=0.05)
        self.fc4 = nn.Linear(256, 128 )
        self.bnorm4 = nn.BatchNorm1d(128 )
        self.dp4 = nn.Dropout(p=0.025)
        self.fc5 = nn.Linear(128 , 346)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dp1(self.bnorm1(x))
        x = F.relu(self.fc2(x))
        x = self.dp2(self.bnorm2(x))
        x = F.relu(self.fc3(x))
        x = self.dp3(self.bnorm3(x))
        x = F.relu(self.fc4(x))
        x = self.dp4(self.bnorm4(x))
        x = F.log_softmax(self.fc5(x))
        return x

In [14]:
def inference(model, loader,device, n_members):
    correct = 0
    for data, label in loader:
        X = Variable(data.view(-1, 13))
        Y = Variable(label.view(-1))
        X = X.to(device)
        Y = Y.to(device)
        model= model.to(device)
        out = model(X)
        pred = out.data.max(1, keepdim=True)[1]
        predicted = pred.eq(Y.data.view_as(pred))
        correct += predicted.sum()
    correct = correct.cpu()
    return correct.numpy() / n_members

class Trainer():
    """ 
    A simple training cradle
    """
    
    def __init__(self, model, optimizer,device, load_path=None):
        self.model = model
        if load_path is not None:
            self.model = torch.load(load_path)
        self.optimizer = optimizer
        self.device = device
            
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

    def run(self, n_epochs):
        print("Start Training...")
        self.metrics = []
        for e in range(n_epochs):
            start_time = time.time()
            epoch_loss = 0
            correct = 0
            for batch_idx, (data, label) in enumerate(train_loader):
                self.optimizer.zero_grad()
                X = Variable(data.view(-1, 13))
                Y = Variable(label.view(-1))
                X = X.to(device)
                Y = Y.to(device)
                self.model= self.model.to(device)
                out = self.model(X)
                pred = out.data.max(1, keepdim=True)[1]
                predicted = pred.eq(Y.data.view_as(pred))
                correct += predicted.sum()
                #loss = F.nll_loss(out, Y)
                criterion = nn.CrossEntropyLoss()
                loss = criterion(out, Y)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.data.item()

            
            total_loss = epoch_loss/train_size
            correct = correct.cpu()
            
            train_error = 1.0 - correct.numpy()/train_size
            train_acc = correct.numpy()/train_size
            val_acc = inference(self.model, val_loader,device, val_size)
            val_error = 1.0 - val_acc
            print("epoch: {0}, loss: {1:.8f}".format(e+1, total_loss))
            print('Train accuracy = ', train_acc)
            print('Validation accuracy = ', val_acc)
            print("--- %s seconds ---" % (time.time() - start_time))
            print("\n===================================\n")
            if e%10 ==0:              
              self.save_model("/content/drive/My Drive/Colab Notebooks/trained_model/trained_model3.pt")
              print("model saved...")
            self.metrics.append(Metric(loss=total_loss, 
                                  train_error=train_error,
                                  val_error=val_error))
         

In [ ]:
##a = b +dafd

In [ ]:
val_size = val.shape[0]

test_size = test.shape[0]

batch_size = 32


In [ ]:
# Training

def init_xavier(m):
    if type(m) == nn.Linear:
        fan_in = m.weight.size()[1]
        fan_out = m.weight.size()[0]
        std = np.sqrt(2.0 / (fan_in + fan_out))
        m.weight.data.normal_(0,std)
#to run on gpu        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
n_epochs = 100

model = PhonesModel()
#load_path_model = "/content/drive/My Drive/Colab Notebooks/trained_model/trained_model2.pt"
# model.load_state_dict(torch.load(load_path_model))
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 0.001)
btrainer = Trainer(model.double(), optimizer,device)
btrainer.run(n_epochs)
#btrainer.save_model('./dropout-batchnorm_optimized_model.pt')

Start Training...


In [ ]:
#t = get_newData(test)
#test_loader = torch.utils.data.DataLoader(t, batch_size=batch_size, shuffle=False)


In [ ]:
#test_acc = inference(model, test_loader,device, test_size)
#print("Test accuracy of model optimizer with: {0:.2f}".format(test_acc * 100))


In [ ]:
### VISUALIZATION ###
def training_plot(metrics):
    plt.figure(1)
    plt.plot([m.loss for m in metrics], 'b')
    plt.title('Training Loss')
    plt.show()

training_plot(btrainer.metrics)

In [ ]:
#btrainer.metrics

In [ ]:
# test.shape

In [ ]:
# out = model(t)

In [ ]:
phones = np.loadtxt("/content/drive/My Drive/Colab Notebooks/Hw3/phones.txt", dtype=str)
phones_labels = np.zeros(346,)
i = 1

while i <346:
  phones_labels[i] = int(phones[i][1])
  i = i +1

phones_labels.shape
phones_labels.size

In [ ]:
# Datasets
class PhonesDatasetTest(Dataset):

  """ Phones dataset."""
  # Initialize your data, etc.
  def __init__(self, x,y):

    self.train = get_newData(x)
    self.train_labels = get_newData(y)
    self.len = len(self.train)
    print(self.train.shape)

    
  def __getitem__(self, index):
    return self.train[index], self.train_labels[index]

  def __len__(self):
    return len(self.train)

  def len():
    return len(self.train)

In [ ]:
#test = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntest.npy',allow_pickle=True)
#test = PhonesDatasetTest(test, phones_labels)
#test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

In [ ]:
def inferenceTest(model, loader,device, n_members):
    correct = 0
    for data, label in loader:
        X = Variable(data.view(-1, 13))
        Y = phones_labels
        Y = torch.from_numpy(Y)
        print(X.numpy().shape)
        print(Y.numpy().shape)
        X = X.to(device)
        Y = Y.to(device)
        model= model.to(device)
        out = model(X)
        pred = out.data.max(1, keepdim=True)[1]
        print(pred)
        predicted = pred.eq(Y.data.view_as(pred))
        correct += predicted.sum()
    correct = correct.cpu()
    return correct.numpy() / n_members


In [ ]:
#inferenceTest(model, test_loader, device,test_size)

In [ ]:
model = PhonesModel()
load_path_model = "/content/drive/My Drive/Colab Notebooks/trained_model/trained_model3.pt"
model.load_state_dict(torch.load(load_path_model))


In [ ]:
test[0][0]

In [ ]:
modelx = model.double()
out = modelx(torch.from_numpy(test[0]))

In [ ]:
out[1000]

In [ ]:
val_labels[1000]